<h1>Welcome to the capstone project where the program returns the best borough to start an italian restaurant in</h1>

starting off by importing all important libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!pip install beautifulsoup4
!pip install requests
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup 
from urllib.request import urlopen
import csv

Solving environment: done

# All requested packages already installed.



web scraping a wikipedia web page containing our data 

In [3]:
web_page = requests.get("https://en.wikipedia.org/wiki/List_of_London_boroughs")
wiki_soup = BeautifulSoup(web_page.text, 'html')

data = []
columns = []
table = wiki_soup.find(class_='wikitable')

In [4]:
for i, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    if (i == 0):
        columns = section
    else:
        data.append(section)

In [5]:
wiki_df = pd.DataFrame(data = data,columns = columns)
wiki_df

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25
1,Barnet,,,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31
2,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23
3,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12
4,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20
5,Camden,Y,,Camden London Borough Council,Labour,"Camden Town Hall, Judd Street",8.40,"229,719",51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ /...,11
6,Croydon,,,Croydon London Borough Council,Labour,"Bernard Weatherill House, Mint Walk",33.41,"372,752",51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ /...,19
7,Ealing,,,Ealing London Borough Council,Labour,"Perceval House, 14-16 Uxbridge Road",21.44,"342,494",51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ /...,13
8,Enfield,,,Enfield London Borough Council,Labour,"Civic Centre, Silver Street",31.74,"320,524",51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ /...,30
9,Greenwich [note 2],Y [note 3],Royal,Greenwich London Borough Council,Labour,"Woolwich Town Hall, Wellington Street",18.28,"264,008",51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E﻿ /...,22


cleaning the data so that we drop all what we will not use and getting the latitude longitude values

In [6]:
london_df = wiki_df.filter(["Borough", "Co-ordinates"], axis=1)
london_df
list = []
list2 = []
for b in london_df["Borough"]:
    list.append(b)
list[0], list[9], list[11] = list[0][:-9], list[9][:-9], list[11][:-9]
london_df["Borough"] = list
london_df["latitude"] = list
london_df["longitude"] = list

for k in london_df["Co-ordinates"]:
    list2.append(k)
for i in range(len(list2)):
    list2[i] = list2[i][47:62]
    list2[i] =  list2[i].replace(';', '')
print(list2)
latitudes__ = []
longitudes__ = []

['51.5607 0.1557', '51.6252 -0.151', '51.4549 0.1505', '51.5588 -0.281', '51.4039 0.0198', '51.5290 -0.125', '51.3714 -0.097', '51.5130 -0.308', '51.6538 -0.079', '51.4892 0.0648', '51.5450 -0.055', '51.4927 -0.233', '51.6000 -0.111', '51.5898 -0.334', '51.5812 0.1837', '51.5441 -0.476', '51.4746 -0.368', '51.5416 -0.102', '51.5020 -0.194', '51.4085 -0.306', '51.4607 -0.116', '51.4452 -0.020', '51.4014 -0.195', '51.5077 0.0469', '51.5590 0.0741', '51.4479 -0.326', '51.5035 -0.080', '51.3618 -0.194', '51.5099 -0.005', '51.5908 -0.013', '51.4567 -0.191', '51.4973 -0.137']


In [7]:
l = []
for i in range(len(list2)):
    list2[i] = list2[i].split(" ")
    l.append(list2[i])

In [8]:
for f in l:
    latitudes__.append(f[0]) 
    longitudes__.append(f[1])
latitudes__ = [float(i) for i in latitudes__]
longitudes__ = [float(i) for i in longitudes__] 

In [9]:
london_df["latitude"] = latitudes__
london_df["longitude"] = longitudes__
london_df = london_df.drop("Co-ordinates", axis=1)

showing first five rows from the cleaned dataframe:

In [10]:
london_df.head()

,Borough,latitude,longitude
0,Barking and Dagenham,51.5607,0.1557
1,Barnet,51.6252,-0.1510
2,Bexley,51.4549,0.1505
3,Brent,51.5588,-0.2810
4,Bromley,51.4039,0.0198


In [11]:
address = 'London'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


<h4>showing a map of London marking the centers of the boroughs</h4>

In [12]:
map_London = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(london_df['latitude'], london_df['longitude'], london_df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_London)  
    
map_London

making an api call to get the venues in london

In [13]:
CLIENT_ID = 'E0XVISM3PSYOOCS3BCDFFBWTDDFBFZ3ROVJM1U4TP2K5ICCR' # your Foursquare ID
CLIENT_SECRET = 'M152I0W3AYM53H1JXURHA1D0LWWQFOL2VXFPGULWG0ZPS1LB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
radius = 1000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

Your credentails:
CLIENT_ID: E0XVISM3PSYOOCS3BCDFFBWTDDFBFZ3ROVJM1U4TP2K5ICCR
CLIENT_SECRET:M152I0W3AYM53H1JXURHA1D0LWWQFOL2VXFPGULWG0ZPS1LB


'https://api.foursquare.com/v2/venues/search?client_id=E0XVISM3PSYOOCS3BCDFFBWTDDFBFZ3ROVJM1U4TP2K5ICCR&client_secret=M152I0W3AYM53H1JXURHA1D0LWWQFOL2VXFPGULWG0ZPS1LB&ll=51.5073219,-0.1276474&v=20180604&radius=1000&limit=100'

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
results = requests.get(url).json()
results

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id,location.crossStreet
0,50aa6e60e4b078585157bbb5,Trafalgar Square Lions,"[{'id': '52e81612bcbc57f1066b79ed', 'name': 'O...",v-1595965650,False,Trafalgar Sq,51.507641,-0.127888,"[{'label': 'display', 'lat': 51.50764064540292...",39,WC2N 5DN,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...",NaN,NaN,NaN
1,4ac518cef964a520f9a520e3,Trafalgar Square,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",v-1595965650,False,Trafalgar Sq,51.507987,-0.128048,"[{'label': 'display', 'lat': 51.507987, 'lng':...",79,WC2N 5DX,GB,London,Greater London,United Kingdom,"[Trafalgar Sq, London, Greater London, WC2N 5D...",NaN,NaN,NaN
2,4d53d735a64560fca1b3b33a,Statue of King Charles I,"[{'id': '52e81612bcbc57f1066b79ed', 'name': 'O...",v-1595965650,False,Trafalgar Sq,51.507344,-0.127720,"[{'label': 'display', 'lat': 51.50734357708428...",5,SW1,GB,Charing Cross,Greater London,United Kingdom,"[Trafalgar Sq, Charing Cross, Greater London, ...",NaN,NaN,NaN
3,4ecfb3860e61efa330295bcf,Crest Of London,"[{'id': '4bf58dd8d48988d128951735', 'name': 'G...",v-1595965650,False,Piccadilly Circus,51.507341,-0.127724,"[{'label': 'display', 'lat': 51.50734145251317...",5,NaN,GB,London,Greater London,United Kingdom,"[Piccadilly Circus, London, Greater London, Un...",NaN,NaN,NaN
4,4c9dc30454c8a1cd0be4864b,Pilgrim Brewery,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",v-1595965650,False,West Street,51.238112,-0.210487,"[{'label': 'display', 'lat': 51.238112, 'lng':...",30516,RH2 9BL,GB,Reigate,Surrey,United Kingdom,"[West Street, Reigate, Surrey, RH2 9BL, United...",NaN,NaN,NaN


In [16]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

no_venues = dataframe_filtered.shape
print("There is {} venues".format(no_venues[0]))

There is 100 venues


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
london_venues = getNearbyVenues(names=london_df['Borough'],
                                   latitudes=london_df['latitude'],
                                   longitudes=london_df['longitude']
                                  )

Barking and Dagenham
Barnet
Bexley
Brent
Bromley
Camden
Croydon
Ealing
Enfield
Greenwich 
Hackney
Hammersmith and Fulham
Haringey
Harrow
Havering
Hillingdon
Hounslow
Islington
Kensington and Chelsea
Kingston upon Thames
Lambeth
Lewisham
Merton
Newham
Redbridge
Richmond upon Thames
Southwark
Sutton
Tower Hamlets
Waltham Forest
Wandsworth
Westminster


In [19]:
london_venues.shape

(1449, 7)

<h3>The venues dataframe</h3>

In [20]:
london_venues.head()

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barking and Dagenham,51.5607,0.1557,Central Park,51.559560,0.161981,Park
1,Barking and Dagenham,51.5607,0.1557,Crowlands Heath Golf Course,51.562457,0.155818,Golf Course
2,Barking and Dagenham,51.5607,0.1557,Robert Clack Leisure Centre,51.560808,0.152704,Martial Arts Dojo
3,Barking and Dagenham,51.5607,0.1557,Beacontree Heath Leisure Centre,51.560997,0.148932,Gym / Fitness Center
4,Barking and Dagenham,51.5607,0.1557,Becontree Heath Bus Station,51.561065,0.150998,Bus Station


In [21]:
london_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Barking and Dagenham,7,7,7,7,7,7
Barnet,4,4,4,4,4,4
Bexley,28,28,28,28,28,28
Brent,79,79,79,79,79,79
Bromley,39,39,39,39,39,39
Camden,76,76,76,76,76,76
Croydon,41,41,41,41,41,41
Ealing,75,75,75,75,75,75
Enfield,54,54,54,54,54,54


In [206]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 208 uniques categories.


In [26]:
k = 0
l = []
for i in london_venues['Venue Category'] == "Italian Restaurant":
    if i == True:
        k = k + 1
print("returned {} Italian restaurant in the city".format(k))

returned 46 Italian restaurant in the city


<h2>dropping all rows that arent italian restaurants</h2>

In [34]:
Italian_london = london_venues.loc[london_venues['Venue Category'] == "Italian Restaurant"]
Italian_london.reset_index(drop = True, inplace = True)
Italian_london

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bexley,51.4549,0.1505,Zizzi,51.455929,0.150555,Italian Restaurant
1,Brent,51.5588,-0.2810,Prezzo,51.556717,-0.282753,Italian Restaurant
2,Brent,51.5588,-0.2810,Zizzi,51.556748,-0.283492,Italian Restaurant
3,Camden,51.5290,-0.1250,Casa Tua,51.527580,-0.123635,Italian Restaurant
4,Camden,51.5290,-0.1250,Albertini Restaurant,51.528757,-0.129138,Italian Restaurant
5,Camden,51.5290,-0.1250,Prezzo,51.527310,-0.130876,Italian Restaurant
6,Croydon,51.3714,-0.0970,Ponte Nuovo,51.370656,-0.100056,Italian Restaurant
7,Ealing,51.5130,-0.3080,Osteria del Portico,51.511354,-0.304877,Italian Restaurant
8,Ealing,51.5130,-0.3080,Pasta Remoli,51.513472,-0.306686,Italian Restaurant
9,Ealing,51.5130,-0.3080,Piccola Italia,51.510843,-0.305551,Italian Restaurant


marking the italian restaurants in the city

In [37]:
#toronto_merged["ClusterLabels"] = toronto_merged["ClusterLabels"].astype(int) 
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon, poi, ven_name in zip(Italian_london['Venue Latitude'], Italian_london['Venue Longitude'], Italian_london['Borough'], Italian_london['Venue']):
    label = folium.Popup(str(poi) +',' + str(ven_name), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

we can see that these are the borough with least amount of restaurants

In [59]:
d = Italian_london.pivot_table(index=['Borough'], aggfunc='size')
df_count = pd.DataFrame(data = d)
df_count = df_count.reset_index()
df_count = df_count.loc[df_count[0] == 1]
df_count

,Borough,0
0,Bexley,1
3,Croydon,1
5,Enfield,1
6,Hackney,1
8,Haringey,1
15,Newham,1


In [73]:
ll = []
ll = df_count["Borough"]
filtered_df = london_df[london_df.Borough.isin(ll)]
filtered_df

,Borough,latitude,longitude
2,Bexley,51.4549,0.1505
6,Croydon,51.3714,-0.0970
8,Enfield,51.6538,-0.0790
10,Hackney,51.5450,-0.0550
12,Haringey,51.6000,-0.1110
23,Newham,51.5077,0.0469


<h2>calculating the distance between each borough and the center of London to decide which borough fits the best</h2>

In [75]:
from math import sin, cos, sqrt, atan2, radians

R = 6373.0

for lat, lon, borough in zip(filtered_df['latitude'], filtered_df['longitude'], filtered_df['Borough']):
    bor = borough
    lat1 = radians(lat)
    lon1 = radians(lon)
    lat2 = radians(latitude)
    lon2 = radians(longitude)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    print("distance between {} and london center equals: {}".format(bor, distance))

distance between Bexley and london center equals: 20.130470519869633
distance between Croydon and london center equals: 15.267168090995826
distance between Enfield and london center equals: 16.63609152210917
distance between Hackney and london center equals: 6.5451053323321435
distance between Haringey and london center equals: 10.37266603537099
distance between Newham and london center equals: 12.084124234743399


<h1>AS SHOWN BY OUR DATA "HACKNEY" IS THE CLOSEST TO "LONDON" center AND FROM the LEAST BOROUGHS WITH ITALIAN RESTAURANTS IN IT</h1>